In [1]:
# Importing libraries

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from tika import parser
import requests
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

# Request PDF URLs

url_2012 = 'https://sso.agc.gov.sg/Act/PDPA2012?ViewType=Pdf'
# url_2012 = input('Enter the URL of the first PDF: ')
response_2012 = requests.get(url_2012)

url_2014 = 'https://sso.agc.gov.sg//SL/PDPA2012-S362-2014?DocDate=20140519&ViewType=Pdf&_=20180520142357'
# url_2014 = input('Enter the URL of the second PDF: ')
response_2014 = requests.get(url_2014)


# Save URL content to PDF locally

with open("sample_2012.pdf",'wb') as f: 
    f.write(response_2012.content)

with open("sample_2014.pdf",'wb') as f: 
    f.write(response_2014.content) 


# Parsing text from PDF using tika

text_2012 = parser.from_file('sample_2012.pdf')
text_2014 = parser.from_file('sample_2014.pdf')

# Stopwords removal using NLTK

stop_words = stopwords.words('english')


# Cleaning text of PDF from the year 2014
# Meanwhile, specific to the document
# Must be generalized for broad usage
# Removal of new lines
# Removal of footers
# Removal of headers
# Removal of headings - text containing 'definitions' text
# Removal of unnecessary text during initial and end pages of the PDF
# Removal of excessively small sentences
# Stopwords removal

sentences_2014 = sent_tokenize(text_2014['content'])
sentences_2014 = [sent.replace('\n', '') for sent in sentences_2014]
sentences_2014 = sentences_2014[15:]
sentences_2014 = [sent.replace('Informal Consolidation – version in force from 2/7/2014', '') for sent in sentences_2014]
sentences_2014 = [sent.replace('S 362/2014', '') for sent in sentences_2014]
sentences_2014 = [sent for sent in sentences_2014 if 'Definitions of this Part' not in sent]
sentences_2014 = sentences_2014[:42]
sentences_2014 = [sent for sent in sentences_2014 if len(sent) > 20]
sentences_2014 = [' '.join(w for w in sent.split() if w not in stop_words) for sent in sentences_2014]

# Cleaning text of PDF from the year 2012
# Meanwhile, specific to the document
# Must be generalized for broad usage
# Removal of new lines
# Removal of footers
# Removal of headers
# Removal of unnecessary text during initial and end pages of the PDF
# Removal of excessively small sentences
# Stopwords removal

sentences_2012 = sent_tokenize(text_2012['content'])
sentences_2012 = [sent.replace('\n', '') for sent in sentences_2012]
sentences_2012 = sentences_2012[74:]
sentences_2012 = [sent.replace('Informal Consolidation – version in force from 2/10/2016', '') for sent in sentences_2012]
sentences_2012 = [sent.replace('26 OF 2012', '') for sent in sentences_2012]
sentences_2012 = sentences_2012[:361]
sentences_2012 = [sent for sent in sentences_2012 if len(sent) > 20]
sentences_2012 = [' '.join(w for w in sent.split() if w not in stop_words) for sent in sentences_2012]

# Using Universal Sentence Encoder from TensorFlowHub
# Computing the similarity of a sample sentence from 2014 PDF to all sentences from 2012 PDF

module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" 
embed = hub.Module(module_url)

similarity_input_placeholder = tf.placeholder(tf.string, shape=(None))
similarity_sentences_encodings = embed(similarity_input_placeholder)
result_list = []
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    sentences_embeddings_2012 = session.run(similarity_sentences_encodings, feed_dict={similarity_input_placeholder: sentences_2012})
    sentences_embeddings_2014 = session.run(similarity_sentences_encodings, feed_dict={similarity_input_placeholder: sentences_2014})
    for i in range(len(sentences_2012)):
        similarity = np.inner(sentences_embeddings_2014[0], sentences_embeddings_2012[i])
        if similarity > 0.8:
            result_list.append(sentences_2012[i])


# Returning a sample sentence from 2014 PDF and its similar sentences from 2012 PDF

print('Sample sentence: {} ########################## Similar sentences: {} ########## {}'.format(sentences_2014[0], result_list[0], result_list[1]))

W0518 12:15:11.904128 15944 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


Instructions for updating:
Colocations handled automatically by placer.


W0518 12:15:28.006524 15944 deprecation.py:323] From C:\Users\Karan.Arya\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\tensorflow\python\ops\control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0518 12:15:31.145538 15944 saver.py:1483] Saver not created because there are no variables in the graph to restore


Sample sentence: These Regulations may cited Personal Data ProtectionRegulations 2014 shall come operation 2nd July 2014. ########################## Similar sentences: This Act may cited Personal Data Protection Act 2012and shall come operation date Minister may, bynotification Gazette, appoint. ########## Notwithstanding provisions Part, anorganisation may use personal data individual collectedbefore appointed day purposes personal datawas collected unless —(a) consent use withdrawn accordance withsection 16; or(b) individual, whether before, appointedday, otherwise indicated organisation doesnot consent use personal data.
